In [1]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as fn
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from numpy.polynomial import chebyshev
from numpy.polynomial.chebyshev import chebgrid2d
from torch.utils.data import Dataset
from torchvision.transforms import transforms
from torch.utils.data import DataLoader

In [2]:
camera_center = np.array([0.15, 0, 0.4]) #We need the coordinates for the camera, screen and unit under test (UUT)
screen_point = np.array([-0.15, 0, 0.4])
screen_normal = np.array([0.15, 0, -0.4])
UUT_resolution = [21,21] #This is the number of pixels in the surface. Make sure each side is an odd integer and they are equal.
UUT_X_range = np.array([-0.02, 0.02])
UUT_Y_range = np.array([-0.02, 0.02])
reference_point = np.array((0, 0, 0)) #Reference point for deflectometry calcs.
reference_normal = np.array((0, 0, 1))
UUT_num_test = 10 #Number of tests
UUT_num_train = 100
terms = 9 #for random number generator
scaling_factor = 0.6 #scale size of unit under test
epoch = 10 #NUmber of epochs
lr = 0.00001 #learning rate. Should be less than 0.1

In [3]:
def generate_surface(UUT_num, Xt, Yt): #This generates a random shape using Chebyshev polynomials
 
    np.random.seed(11)
    C = np.random.rand(UUT_num, terms, terms)
    UUT = np.empty([UUT_num, Xt.shape[0], Yt.shape[0]])
    for i in range(UUT_num):
        UUT[i] = chebgrid2d(Xt, Yt, C[i])
    return C, UUT

In [4]:
def normalization_slopes(C, Xt, Yt, dtdx, dtdy): #This finds the slopes of the generated surface
  
    dx = np.empty([C.shape[0], Xt.shape[0], Yt.shape[0]])
    dy = np.empty(dx.shape)
    for i in range(C.shape[0]):
        dxc = chebyshev.chebder(C[i], axis=0)
        dx[i] = chebgrid2d(Xt, Yt, dxc) * dtdx
        dyc = chebyshev.chebder(C[i], axis=1)
        dy[i] = chebgrid2d(Xt, Yt, dyc) * dtdy

    N = np.sqrt(1 + np.square(dx) + np.square(dy))
    n = np.stack((-dx / N, -dy / N, 1 / N), axis=-1)  
    return dx, dy, n

In [5]:
def reflect(camera_center, X, Y, UUT, n, screen_point, screen_normal): #reflection from mirror

    Xg, Yg = np.meshgrid(X, Y)
    Xg = Xg.T  
    Yg = Yg.T
    UUT = UUT / 1000  
    UUT_3D = np.stack((np.expand_dims(Xg, 0).repeat(UUT.shape[0], axis=0),
                       np.expand_dims(Yg, 0).repeat(UUT.shape[0], axis=0)
                       , UUT), axis=-1)
    I = UUT_3D - camera_center  
    temp = (I * n).sum(axis=-1)
    Reflect = I - 2 * np.stack((temp, temp, temp), axis=-1) * n  

    t1 = ((screen_point - UUT_3D) * screen_normal).sum(axis=-1)
    t2 = (screen_normal * Reflect).sum(axis=-1)
    t = t1 / t2
    reflect_screen = np.stack((t, t, t), axis=-1) * Reflect + UUT_3D  
    return I, Reflect, reflect_screen

In [6]:
def reference(camera_center, Incident, reflect_screen, reference_point, reference_normal): #reference coordinates for calcs.
    
    t1 = ((reference_point - camera_center) * reference_normal).sum()
    t2 = (reference_normal * Incident).sum(axis=-1)
    t = t1 / t2
    reference_surface = np.stack((t, t, t), axis=-1) * Incident + camera_center  

    
    dr2c = np.sqrt(np.square(reference_surface - camera_center).sum(axis=-1))
    dr2s = np.sqrt(np.square(reference_surface - reflect_screen).sum(axis=-1))
    xr = reference_surface[..., 0]  
    yr = reference_surface[..., 1]
    zr = reference_surface[..., 2]

    xc = camera_center[..., 0]  
    yc = camera_center[..., 1]
    zc = camera_center[..., 2]

    xs = reflect_screen[..., 0] 
    ys = reflect_screen[..., 1]
    zs = reflect_screen[..., 2]

    denominator = (zc - zr) / dr2c + (zs - zr) / dr2s
    reference_slope_x = ((xr - xc) / dr2c + (xr - xs) / dr2s) / denominator
    reference_slope_y = ((yr - yc) / dr2c + (yr - ys) / dr2s) / denominator

    return reference_surface, reference_slope_x, reference_slope_y

In [7]:
class SurfaceDataset(Dataset): #used for generating surface
    def __init__(self, UUT_resolution, UUT_num, UUT_X_range, UUT_Y_range, camera_center, screen_point,
                 screen_normal, reference_point, reference_normal, transform=None):
        self.transform = transform
        self.UUT_resolution = UUT_resolution
        self.UUT_num = UUT_num
        self.UUT_X_range = UUT_X_range
        self.UUT_Y_range = UUT_Y_range
        self.camera_center = camera_center
        self.screen_point = screen_point
        self.screen_normal = screen_normal

        X = np.linspace(UUT_X_range[0], UUT_X_range[1], UUT_resolution[0])
        Y = np.linspace(UUT_Y_range[0], UUT_Y_range[1], UUT_resolution[1])
        Xt = (UUT_X_range.sum() - 2 * X) / (UUT_X_range[0] - UUT_X_range[1]) * scaling_factor
        Yt = (UUT_Y_range.sum() - 2 * Y) / (UUT_Y_range[0] - UUT_Y_range[1]) * scaling_factor

        dtdx = scaling_factor * -2 / (UUT_X_range[0] - UUT_X_range[1]) / 1000
        dtdy = scaling_factor * -2 / (UUT_Y_range[0] - UUT_Y_range[1]) / 1000
        self.C, self.UUT = generate_surface(UUT_num, Xt, Yt)
        self.dx, self.dy, self.n = normalization_slopes(self.C, Xt, Yt, dtdx, dtdy)
        self.Incident, self.Reflect, self.reflect_screen = reflect(camera_center, X, Y, self.UUT, self.n, screen_point,
                                                                   screen_normal)

        self.reference_surface, self.reference_slope_x, self.reference_slope_y = reference(camera_center, self.Incident,
                                                                               self.reflect_screen, reference_point,
                                                                               reference_normal)
        self.reference_slope = np.stack((self.reference_slope_x, self.reference_slope_y), axis=-1)

    def __getitem__(self, index):
        slope, UUT = self.reference_slope[index], self.UUT[index]
        if self.transform is not None:
            slope = self.transform(slope)

        return slope, UUT

    def __len__(self):
        return len(self.reference_slope)


In [8]:
class deflectometry_model(nn.Module): #Convolutional neural network
    def __init__(self, width=None):
        super(deflectometry_model, self).__init__()
        if width is None:
            width = [32, 128, 256, 1024] #Layers follow below. 
        self.conv1 = nn.Conv2d(in_channels=2, out_channels=width[0], kernel_size=7, stride=1, padding=3, bias=False,
                               padding_mode="replicate")
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(width[0], width[1], 7, 1, 3, bias=False, padding_mode="replicate")
        self.pool2 = nn.AvgPool2d(2, 2)
        
        self.conv3 = nn.Conv2d(width[1], width[2], 7, 1, 3, bias=False, padding_mode="replicate")
        self.pool3 = nn.AvgPool2d(2, 2)
        
        self.conv4 = nn.Conv2d(width[2], width[3], 1, 1, bias=False, padding_mode="replicate")

        
        self.conv5 = nn.Conv2d(width[3] + width[2], width[2], 7, 1, 3, bias=False, padding_mode="replicate")
        self.conv6 = nn.Conv2d(width[2] + width[1], width[1], 7, 1, 3, bias=False, padding_mode="replicate")
        self.conv7 = nn.Conv2d(width[1] + width[0], width[0], 7, 1, 3,  bias=False, padding_mode="replicate")
        self.conv8 = nn.Conv2d(width[0], 1, 7, 1, 3, bias=False, padding_mode="replicate")

    def forward(self, x):
       
        x = x.float()
        conv1 = self.conv1(x)
        x = self.pool1(conv1)
        conv2 = self.conv2(x)
        x = self.pool2(conv2)
        conv3 = self.conv3(x)
        x = self.pool3(conv3)
        x = self.conv4(x)
        x = fn.resize(x, size=[conv3.shape[2], conv3.shape[3]]) 
        x = torch.cat((x, conv3), 1)  
        x = self.conv5(x)  # conv5
        x = fn.resize(x, size=[conv2.shape[2], conv2.shape[3]])  
        x = torch.cat((x, conv2), 1)  
        x = self.conv6(x)  # conv6
        x = fn.resize(x, size=[conv1.shape[2], conv1.shape[3]])  
        x = torch.cat((x, conv1), 1) 
        x = self.conv7(x)
        x = self.conv8(x)
        x = torch.squeeze(x,dim=1)

        return x

In [9]:
 transform = transforms.Compose([
        transforms.ToTensor()
    ]) #Later on, we'll need to convert data to the tensor type for PyTorch. The code below creates the training and testing data.
train_data =  SurfaceDataset(UUT_resolution, UUT_num_train, UUT_X_range, UUT_Y_range, camera_center, screen_point,
                     screen_normal, reference_point, reference_normal, transform)
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=False, num_workers=0)
test_data =  SurfaceDataset(UUT_resolution, UUT_num_test, UUT_X_range, UUT_Y_range, camera_center, screen_point,
                     screen_normal, reference_point, reference_normal, transform)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False, num_workers=0)
#Create training and testing data

In [10]:
model = deflectometry_model()
model = model.to("cpu")

loss = nn.MSELoss() #Measures the Mean squared error
loss = loss.to("cpu")
optim = torch.optim.Adam(model.parameters(), lr=lr)
for i in range(epoch): #goes through and trains model
    model.train()
    for data in train_loader:
        reference_slope, UUT = data
        reference_slope = reference_slope.float()
        UUT = UUT.float()
        reference_slope = reference_slope.to("cpu")
        UUT = UUT.to("cpu")
        pred_UUT = model(reference_slope)
        result_loss = loss(pred_UUT, UUT)
        optim.zero_grad()
        result_loss.backward()
        optim.step()

    
    model.eval() #evaluates
    total_test_loss = 0
    with torch.no_grad():
        for data in test_loader:
            reference_slope, UUT = data
            reference_slope = reference_slope.float()
            UUT = UUT.float()
            reference_slope = reference_slope.to("cpu")
            UUT = UUT.to("cpu")
            pred_UUT = model(reference_slope)
            result_loss = loss(pred_UUT, UUT)
            total_test_loss += result_loss
    print("Epoch number {}, Loss Rate (True - Estimated): {}".format(i, total_test_loss))
    pred_UUT_array = np.array(pred_UUT)
    UUT_array = np.array(UUT)
    reference_slope = np.array(reference_slope)

Epoch number 0, Loss Rate (True - Estimated): 2.3951196670532227
Epoch number 1, Loss Rate (True - Estimated): 2.3518271446228027
Epoch number 2, Loss Rate (True - Estimated): 2.3052451610565186
Epoch number 3, Loss Rate (True - Estimated): 2.2541444301605225
Epoch number 4, Loss Rate (True - Estimated): 2.199474811553955
Epoch number 5, Loss Rate (True - Estimated): 2.1454391479492188
Epoch number 6, Loss Rate (True - Estimated): 2.0952017307281494
Epoch number 7, Loss Rate (True - Estimated): 2.0436508655548096
Epoch number 8, Loss Rate (True - Estimated): 1.9863991737365723
Epoch number 9, Loss Rate (True - Estimated): 1.9246760606765747
